# Agent

> RL-based Language model Agent

In [ ]:
#| default_exp agent

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from typing import Callable, Tuple, Optional

import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Categorical

from torchtyping import TensorType

### Agent (The RL-based language model)

In [ ]:
#| export
class Agent(nn.Module):
    "The RL-based language model."
    def __init__(
        self,
        model: Callable # a pre-trained `transformers` model
    ):
        super().__init__()
        n_embd = model.config.n_embd
        self.eos_token_id = model.config.eos_token_id

        self.policy_network = model        
        self.value_network = nn.Sequential(
            nn.Linear(n_embd, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Tanh()
        )
    
    def get_value(
        self, hidden_state: TensorType["batch_size", "seq_len", "n_embd"]
    ) -> TensorType["batch_size", 1]:
        """Get value from the value network."""
        return self.value_network(hidden_state)[:, -1, :]

    def generate(
        self,
        input_ids: TensorType["batch_size", "seq_len"],
        attention_mask: Optional[TensorType["batch_size", "seq_len"]] = None,
        **kwargs
    ) -> TensorType["batch_size", "seq_len"]:
        output = self.policy_network.generate(
            input_ids=input_ids, attention_mask=attention_mask, **kwargs
        )
        return output
    
    def forward(
        self,
        input_ids: TensorType["batch_size", "seq_len"],
        attention_mask: Optional[TensorType["batch_size, seq_len"]] = None
    ) -> Tuple[
        TensorType["batch_size", "seq_len", "vocab_size"],
        TensorType["batch_size", "seq_len", "vocab_size"],
        TensorType["batch_size", "seq_len"],
        TensorType["batch_size", 1]
    ]: # action, logprobs, entropy, value
        """_summary_"""
        if attention_mask is None:
            base_output = self.policy_network(
                input_ids,
                output_hidden_states=True,
            )
        else:
            base_output = self.policy_network(
                input_ids, attention_mask=attention_mask,
                output_hidden_states=True,
            )
        
        last_hidden_state = base_output.hidden_states[-1]
        
        # takes the logit of the last token
        # for each sequence in the batch
        logits = base_output.logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
                                
        action_dist = Categorical(probs=probs)
        action = action_dist.sample()
        entropy = action_dist.entropy()
        logprobs = action_dist.log_prob(action)
        
        # predicted reward value
        value = self.get_value(last_hidden_state).squeeze(-1)
        
        return action, logprobs, entropy, value

In [ ]:
show_doc(Agent)

---

### Agent

>      Agent (model:Callable)

The RL-based language model.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| model | Callable | a pre-trained `transformers` model |

In [ ]:
show_doc(Agent.forward)

---

### Agent.forward

>      Agent.forward (input_ids:typing.Annotated[torch.Tensor,{'__torchtyping__'
>                     :True,'details':('batch_size','seq_len',),'cls_name':'Tens
>                     orType'}], attention_mask:Optional[Annotated[torch.Tensor,
>                     {'__torchtyping__':True,'details':('batch_size,seq_len',),
>                     'cls_name':'TensorType'}]]=None)

_summary_

### Agent Objective

Equation 2 in the paper https://arxiv.org/abs/2203.02155

$\begin{aligned} \operatorname{objective~}(\phi)= & E_{(x, y) \sim D_{\pi_\phi^{\mathrm{RL}}}}\left[r_\theta(x, y)-\beta \log \left(\pi_\phi^{\mathrm{RL}}(y \mid x) / \pi^{\mathrm{SFT}}(y \mid x)\right)\right]+ \\ & \gamma E_{x \sim D_{\text {pretrain }}}\left[\log \left(\pi_\phi^{\mathrm{RL}}(x)\right)\right]\end{aligned}$

In [ ]:
#| export
class AgentObjective(nn.Module):
    """Agent objective."""
    def __init__(
        self,
        model: Callable, # the language model
        sft_model: Callable, # the reference model
        reward_model: Callable, # the reward model
        gamma: float,
        beta: float
    ):
        super().__init__()
        self.model = model
        self.sft_model = sft_model
        self.reward_model = reward_model
        self.gamma = gamma
        self.beta = beta
        
    def forward(
        self,
        input_ids: TensorType["batch_size", "seq_len"],
        attention_mask: TensorType["batch_size", "seq_len"]
    ) -> TensorType[1]: # A scalar objective value
        """Calculate the objective value given the input ids and attention mask."""
        model_logits = self.model(input_ids, attention_mask)
        
        model_dist = F.softmax(model_logits, dim=-1)
        
        sft_logits = self.sft_model(input_ids, attention_mask)
        sft_dist = F.softmax(sft_logits, dim=-1)
        
        reward_score = self.reward_model(input_ids, attention_mask)
        
        ratio = torch.log(model_dist / sft_dist)
        
        # compute the coherent of the generated text
        coherent = torch.log(model_dist)
        
        objective = (reward_score - self.beta*ratio).mean() + self.gamma * coherent.mean()
        
        return objective

In [ ]:
show_doc(AgentObjective)

---

### AgentObjective

>      AgentObjective (model:Callable, sft_model:Callable,
>                      reward_model:Callable, gamma:float, beta:float)

Agent objective.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| model | Callable | the language model |
| sft_model | Callable | the reference model |
| reward_model | Callable | the reward model |
| gamma | float |  |
| beta | float |  |

In [ ]:
show_doc(AgentObjective.forward)

---

### AgentObjective.forward

>      AgentObjective.forward (input_ids:typing.Annotated[torch.Tensor,{'__torch
>                              typing__':True,'details':('batch_size','seq_len',
>                              ),'cls_name':'TensorType'}], attention_mask:typin
>                              g.Annotated[torch.Tensor,{'__torchtyping__':True,
>                              'details':('batch_size','seq_len',),'cls_name':'T
>                              ensorType'}])

Calculate the objective value given the input ids and attention mask.

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| input_ids | Annotated |  |
| attention_mask | Annotated |  |
| **Returns** | **Annotated** | **a scalar objective value** |